In [1]:
from pathlib import Path
import utils
from typing import Callable
import scipy
from scipy import signal as sgn
import seaborn

In [2]:
def filtered_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 8
    f_high = 40
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    return Sxx


def naif_fft_features(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    fft = np.fft.rfft(signal_matrix)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    return Sxx

def generate_df(data,features, chanels): 
    #le paso la data resultante de la extraccion de features, los nombres de los features en una lista y de 
    #los canales
    features_name=[]
    for ch in chanels:
        for feature in features:
            features_name.append(feature + 'ch ' + str (ch))
    return pd.DataFrame(data, columns=features_name)

In [3]:
class BCIDataset():
    def __init__(
        self, 
        csvs_path, 
        subject: str = 'all', 
        session: str = 'all',
        channel: str = 'all', 
        overlapping_fraction: float = 1/3, 
        window_size: int = 900,
        feature_extractor: Callable = naif_fft_features
    ):
        '''
        Object containing all examples from a time series from the dataset.
        Args:
            csvs_path (str): path al directorio donde estan los csv de los datos
            subject (str): sujeto a estudiar.
                Si se indica 'all' el dataset final generado tendrá ejemplos de todos los sujetos
            session (str): sesión a estudiar del sujeto seleccionado.
                si se indica 'all'  el dataset final generado tendrá ejemplos de todas las sesiones
            channel (str): 'ch0', 'ch1', 'ch2', 'ch3'. Si se indica 'all', los ejemplos serán la
                concatenación de los 4 canales.
            overlapping_fraction (float): porcentaje de desplazamiento de la "ventana" que hace el ejemplo
            window_size (int): tamaño de la ventana de tiempo que hace a un ejemplo (en muestras).
            feature_extractor (func): Función de extracción de features, le ingresa un arreglo
                (ejemplos en las filas, muestras en las columnas) y devuelve un arreglo (ejemplos en
                las filas y features en las columnas)
        '''
        self.csvs_path = Path(csvs_path)
        self.channel = channel
        self.parts = int(1 / overlapping_fraction)
        self.fraction = 1 / self.parts
        self.ws = window_size
        self.subject = subject
        self.session = session
        self.channels = ['ch0','ch1','ch2','ch3']
        self.feature_extractor = feature_extractor
        self.complete_dataset = utils.read_all_datasets(self.csvs_path)
        
        self.complete_examples_signal, self.complete_examples_features, \
        self.complete_labels, self.complete_metadata = \
            self.generate_examples()
        
    def generate_examples(self):
        # For each session generate the signal examples,
        # the feature extraction examples and labels arrays
        complete_examples_signal, complete_examples_features = [], []
        complete_labels, complete_metadata = [], []
        
        for subject in self.complete_dataset.subject.unique():
            # Just load the selected subject
            if self.subject != 'all' and subject != self.subject:
                continue
            sessions = \
                self.complete_dataset.loc[self.complete_dataset.subject==subject].session.unique()
            for session in sessions:
                # Just load the selected session
                if self.session != 'all' and session != self.session:
                    continue
                
                print(f'Processing subject: {subject} - session: {session}...')
                
                # Generate a subset of the dataset only with the desired rows
                selection = self.complete_dataset.loc[
                    (self.complete_dataset.subject==subject) &
                    (self.complete_dataset.session==session)
                ]
                
                # Standarize length of the array to a multiple to window size
                labels = selection.label.values
                n_rows = labels.shape[0] // self.ws
                labels = labels[: n_rows * self.ws]

                # Generate examples from the signal
                n_examples = n_rows * self.parts - (self.parts - 1)
                examples = np.empty((n_examples, self.ws))
                most_frec_labels = np.empty((n_examples, self.ws))
                times = np.empty((n_examples, self.ws))
                
                concat_ch_examples_signal, concat_ch_examples_features = [], []

                for k, ch in enumerate(self.channels):
                    # Use the four channels or just one
                    if self.channel != 'all' and self.channel != ch:
                        continue
                    
                    # Standarize length of the signal to a multiple to window size
                    signal = selection[ch].values
                    signal = signal[: n_rows * self.ws]
                    time = selection.time.values
                    time = time[: n_rows * self.ws]

                    # Increase the number of examples by overlapping the windows
                    for part in range(self.parts):
                        
                        # Find the place in the output array for each example
                        position = np.arange(part, n_examples, self.parts)
                        #position = position if part == 0 else position[:-part]

                        # Crop the signal according to the window size and overlap
                        start = int(self.ws / self.parts) * part
                        end = -int(self.ws - (self.ws / self.parts) * part)
                        end = end if part!=0 else signal.shape[0]
                        subset_signal = signal[start:end]
                        subset_labels = labels[start:end]
                        subset_times = time[start:end]

                        # Generate the examples
                        n_rows_ = int(subset_signal.shape[0]/self.ws)
                        examples[position, :] = subset_signal.reshape((n_rows_, self.ws))
                        most_frec_labels[position, :] = subset_labels.reshape((n_rows_, self.ws))
                        times[position, :] = subset_times.reshape((n_rows_, self.ws))
                    
                    # Obtain most frequent label
                    labels_ = scipy.stats.mode(most_frec_labels, axis=1).mode
                    labels_temp = scipy.stats.mode(most_frec_labels, axis=1).count
                    pureness = labels_temp == self.ws
                    # Get first and last time of the window
                    times_ = np.asarray([np.min(times, axis=1), np.max(times, axis=1)]).T
                    # Extract features
                    features = self.feature_extractor(examples)

                    concat_ch_examples_signal.append(examples.copy())
                    concat_ch_examples_features.append(features)

                concat_ch_examples_signal = np.concatenate(concat_ch_examples_signal, axis=1)
                concat_ch_examples_features = np.concatenate(concat_ch_examples_features, axis=1)

                complete_examples_signal.append(concat_ch_examples_signal)
                complete_examples_features.append(concat_ch_examples_features)
                complete_labels.append(labels_)
                lt = len(times_)
                metadata_ = np.concatenate(
                    [pureness, times_, np.repeat(subject, lt)[:,None], np.repeat(session, lt)[:,None]],
                    axis=1
                )
                complete_metadata.append(metadata_)

        complete_examples_signal = np.concatenate(complete_examples_signal)
        complete_examples_features = np.concatenate(complete_examples_features)
        complete_labels = np.concatenate(complete_labels)
        complete_metadata = np.concatenate(complete_metadata)
        
        return complete_examples_signal, complete_examples_features, complete_labels, complete_metadata
    
    def __len__(self):
        return self.complete_examples_signal.shape[0]
    
    def __getitem__(self, idx):
        return {
            'signal': self.complete_examples_signal[idx,:],
            'features': self.complete_examples_features[idx,:],
            'label': self.complete_labels[idx,:],
            'metadata': self.complete_metadata[idx,:]
        }

    def get_X_signal(self):
        return self.complete_examples_signal

    def get_X_features(self):
        return self.complete_examples_features

    def get_Y(self):
        return self.complete_labels

    def get_metadata(self):
        return self.complete_metadata

In [4]:
def calc_max_filtered(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 9
    f_high = 22
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)
    
    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    maximum = np.argmax(Sxx, axis=1)
  
    item=freqs[maximum]
    return item.reshape((signal_matrix.shape[0], 1))

In [5]:
def filtered_range(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 9
    f_high = 22
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    index_low=np.where(freqs>=10)[0][0]

    index_high=np.where(freqs>=35)[0][0]

    
    return Sxx[:,index_low:index_high]

In [6]:
def filtered_pca(signal_matrix):
    N = signal_matrix.shape[1]
    dt = 1/200
    T = N*dt
    sf = 200
    Q = 30
    f_notch = 50
    b_notch, a_notch = sgn.iirnotch(w0=f_notch, Q=Q, fs=sf)
    sig_notch = sgn.filtfilt(b_notch, a_notch, signal_matrix, axis=1)

    #Ahora creamos el filtro pasabanda Butterworth
    f_nq = sf/2
    f_low = 10
    f_high = 35
    order = 4
    b_band, a_band = sgn.iirfilter(
        N=order, Wn=[f_low/f_nq, f_high/f_nq], btype="bandpass", ftype="butter"
    )
    sig_filt = sgn.filtfilt(b_band, a_band, sig_notch, axis=1)

    fft = np.fft.rfft(sig_filt)
    Sxx = np.real(((2*dt**2)/T)*fft*fft.conj())
    freqs = np.fft.rfftfreq(n=N, d=dt)
    
    index_low=np.where(freqs>=10)[0][0]

    index_high=np.where(freqs>=35)[0][0]    
    
    from sklearn.decomposition import PCA
    pca = PCA(n_components=3)
    pca.fit(Sxx[:,index_low:index_high])
    proyected_features_freq = pca.transform(Sxx[:,index_low:index_high])
    
    return proyected_features_freq.reshape((signal_matrix.shape[0], 3))

In [7]:
csvs_path = '/home/ariel/gabriel/diplo_de_datos/mentoria_BCI/tp3'

In [62]:
# dataset con los valores de frecuencia donde aparece la amplitud máxima
dataset_filtered_max = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=calc_max_filtered)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [63]:
# dataset con la senal filtrada y sólo teniendo el rango de frecuencias de interés
dataset_filtered_range = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=filtered_range)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# dataset con la senal filtrada y aplicando pca
dataset_filtered_pca = BCIDataset(csvs_path,
                         window_size=750,
                        feature_extractor=filtered_pca)

Processing subject: MA - session: 1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
df_max_freq=generate_df(dataset_filtered_max.complete_examples_features,['max freq '], [0,1,2,3])

<IPython.core.display.Javascript object>

In [65]:

N=dataset_filtered_range.complete_examples_signal.shape[1]
dt = 1/200
freqs_all = np.fft.rfftfreq(n=int(N/4), d=dt)
index_low=np.where(freqs_all>=10)[0][0]
index_high=np.where(freqs_all>=35)[0][0]
name= [str(i) for i in list(freqs_all[index_low:index_high].round(2))]
df_range_freq=pd.DataFrame(dataset_filtered_range.complete_examples_features,columns=name+name+name+name)

df_range_freq['freq_label_1_mean_log']=np.log(df_range_freq[['12.0','12.27','12.53','12.8','13.07']].mean(axis=1))
df_range_freq['freq_label_2_mean_log']=np.log(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].mean(axis=1))

df_range_freq['freq_label_1_median_log']=np.log(np.median(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_median_log']=np.log(np.median(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_sum_log'] = np.log(df_range_freq[['12.0','12.27','12.53','12.8','13.07']].sum(axis=1)/4)
df_range_freq['freq_label_2_sum_log'] = np.log(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].sum(axis=1)/4)

df_range_freq['freq_label_1_quantile_log']=np.log(np.quantile(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],0.9,axis=1))
df_range_freq['freq_label_2_quantile_log']=np.log(np.quantile(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],0.9,axis=1))

df_range_freq['freq_label_1_std_log']=np.log(np.std(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_std_log']=np.log(np.std(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_var_log']=np.log(np.var(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_var_log']=np.log(np.var(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_entropy_log']=np.log(scipy.stats.entropy(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1))
df_range_freq['freq_label_2_entropy_log']=np.log(scipy.stats.entropy(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1))

df_range_freq['freq_label_1_skew']=scipy.stats.skew(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_skew']=scipy.stats.skew(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)

df_range_freq['freq_label_1_kurtosis']=scipy.stats.kurtosis(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_kurtosis']=scipy.stats.kurtosis(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)


#features sin log

df_range_freq['freq_label_1_mean']=df_range_freq[['12.0','12.27','12.53','12.8','13.07']].mean(axis=1)
df_range_freq['freq_label_2_mean']=df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].mean(axis=1)

df_range_freq['freq_label_1_median']=np.median(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_median']=np.median(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)

df_range_freq['freq_label_1_sum'] = df_range_freq[['12.0','12.27','12.53','12.8','13.07']].sum(axis=1)/4
df_range_freq['freq_label_2_sum'] = df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']].sum(axis=1)/4

df_range_freq['freq_label_1_quantile']=np.quantile(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],0.9,axis=1)
df_range_freq['freq_label_2_quantile']=np.quantile(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],0.9,axis=1)

df_range_freq['freq_label_1_std']=np.std(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_std']=np.std(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)

df_range_freq['freq_label_1_var']=np.var(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_var']=np.var(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)

df_range_freq['freq_label_1_entropy']=scipy.stats.entropy(df_range_freq[['12.0','12.27','12.53','12.8','13.07']],axis=1)
df_range_freq['freq_label_2_entropy']=scipy.stats.entropy(df_range_freq[[ '16.0','16.27', '16.53', '16.8','17.07']],axis=1)







array_freqs_label_1=df_range_freq[['12.0','12.27','12.53','12.8','13.07']].to_numpy()
array_freqs_label_2=df_range_freq[['16.0','16.27', '16.53', '16.8','17.07']].to_numpy()
#Hago una suma ponderada de los 4 canales en cjto. tener en cuenta que primero aparecen los valores 
#de cada frecuencia repetida por los cuatro canales
sum_weighted_1_log=np.log(np.dot(array_freqs_label_1,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4]))

sum_weighted_2_log=np.log(np.dot(array_freqs_label_2,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4]))

sum_weighted_1=np.dot(array_freqs_label_1,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4])

sum_weighted_2=np.dot(array_freqs_label_2,[0.1/4,0.1/4,0.1/4,0.1/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.4/4,0.4/4,0.4/4,0.4/4,
                                 0.2/4,0.2/4,0.2/4,0.2/4,
                                 0.1/4,0.1/4,0.1/4,0.1/4])

df_range_freq['sum_weighted_label_1']= sum_weighted_1.tolist()
df_range_freq['sum_weighted_label_2']= sum_weighted_2.tolist()
df_range_freq['sum_weighted_label_1_log']= sum_weighted_1_log.tolist()
df_range_freq['sum_weighted_label_2_log']= sum_weighted_2_log.tolist()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
df_pca_freq=generate_df(dataset_filtered_pca.complete_examples_features,['pca 0 ', 'pca 1 ','pca 2 '], [0,1,2,3])


<IPython.core.display.Javascript object>

In [66]:
labels = pd.DataFrame(dataset_filtered_max.complete_labels,columns=['labels'])
metadata_all = pd.DataFrame(dataset_filtered_max.complete_metadata, columns = ['purness', 't_start', 't_end', 
                                                                      'subject', 'session'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
df_max_freq['max_freq_mean'] = df_max_freq.mean(axis=1)
df_max_freq['max_freq_mode'] = scipy.stats.mode(df_max_freq,axis=1)[0]
df_max_freq['max_freq_median']=np.median(df_max_freq,axis=1)
df_max_freq['max_freq_quantile']=np.quantile(df_max_freq,0.9,axis=1)
df_max_freq['max_freq_std']=np.std(df_max_freq,axis=1)
df_max_freq['max_freq_var']=np.var(df_max_freq,axis=1)
df_max_freq['max_freq_entropy'] = scipy.stats.entropy(df_max_freq,axis=1)
df_max_freq['max_freq_skew'] = scipy.stats.skew(df_max_freq,axis=1)

max_freq_with_labels = df_max_freq.merge(labels,left_index=True,right_index=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
freq_range_with_labels = df_range_freq.merge(labels,left_index=True,right_index=True)


In [69]:
pca_with_labels = df_pca_freq.merge(labels,left_index=True,right_index=True)


In [70]:
best_features=df_max_freq.merge(df_range_freq[['freq_label_1_mean','freq_label_2_mean','freq_label_1_sum','freq_label_2_sum',
          'freq_label_1_median','freq_label_2_median','freq_label_1_quantile','freq_label_2_quantile'
                                 ,'sum_weighted_label_1','sum_weighted_label_2','freq_label_1_std',
                                 'freq_label_2_std','freq_label_1_var','freq_label_2_var','freq_label_1_entropy',
                                 'freq_label_2_entropy','freq_label_1_skew','freq_label_2_skew','freq_label_1_kurtosis',
                                 'freq_label_2_kurtosis',
                                'freq_label_1_mean_log','freq_label_2_mean_log','freq_label_1_sum_log','freq_label_2_sum_log',
          'freq_label_1_median_log','freq_label_2_median_log','freq_label_1_quantile_log','freq_label_2_quantile_log'
                                 ,'sum_weighted_label_1_log','sum_weighted_label_2_log','freq_label_1_std_log',
                                 'freq_label_2_std_log','freq_label_1_var_log','freq_label_2_var_log','freq_label_1_entropy_log',
                                 'freq_label_2_entropy_log']],left_index=True,right_index=True)
best_features["purness"]=metadata_all["purness"].replace({'True':1,'False':0})
best_features = best_features.merge(labels,left_index=True,right_index=True)
best_features


,max freq ch 0,max freq ch 1,max freq ch 2,max freq ch 3,max_freq_mean,max_freq_mode,max_freq_median,max_freq_quantile,max_freq_std,max_freq_var,...,sum_weighted_label_1_log,sum_weighted_label_2_log,freq_label_1_std_log,freq_label_2_std_log,freq_label_1_var_log,freq_label_2_var_log,freq_label_1_entropy_log,freq_label_2_entropy_log,purness,labels
0,13.066667,14.933333,14.666667,13.066667,13.933333,13.066667,13.500000,14.773333,7.630563e-01,17.499774,...,-1.150893,-0.440725,-0.631612,-0.705486,-1.263224,-1.410973,0.881121,0.893487,1,99.0
1,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,1.776357e-15,20.480000,...,0.401673,0.068644,-0.259133,-0.287339,-0.518265,-0.574678,1.040429,1.024529,1,99.0
2,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,0.000000e+00,24.448176,...,0.437796,0.984072,-0.301823,0.317052,-0.603647,0.634104,1.056579,1.018070,1,99.0
3,15.466667,15.466667,16.533333,16.266667,15.933333,15.466667,15.700000,16.373333,4.129223e-01,23.842956,...,0.640049,0.988608,0.021961,0.055607,0.043921,0.111214,1.036543,1.064096,1,99.0
4,10.133333,18.933333,10.133333,10.133333,12.333333,10.133333,10.133333,14.973333,3.048409e+00,16.376103,...,-0.600641,-0.537895,-0.849648,-0.882854,-1.699296,-1.765708,0.954192,0.991516,1,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,13.333333,11.200000,11.200000,13.600000,12.333333,11.200000,11.766667,13.440000,9.974715e-01,13.410402,...,0.702530,0.358629,-0.057086,-0.139515,-0.114171,-0.279030,1.048016,1.024135,1,1.0
2610,13.600000,19.733333,12.533333,13.866667,14.933333,12.533333,13.733333,16.853333,2.298074e+00,19.942456,...,0.598037,0.627259,0.131704,0.206982,0.263408,0.413963,1.000522,0.966982,1,1.0
2611,13.866667,16.800000,13.866667,13.866667,14.600000,13.866667,13.866667,15.480000,1.016136e+00,18.945899,...,0.230609,0.345996,0.425414,0.090320,0.850828,0.180639,0.862530,0.908462,1,1.0
2612,12.533333,15.733333,12.533333,18.666667,14.866667,12.533333,13.700000,16.906667,2.144199e+00,19.617688,...,0.579868,-0.294638,0.402725,-0.394556,0.805450,-0.789111,0.836923,0.995172,1,1.0


### pruebo con features sin log

In [72]:
X, y = best_features.iloc[:, :-1], best_features.labels

In [73]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y) # se queda con el 10% para validación
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train) # del 90% hace 20% para train y 80% para test

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [74]:
y_train.value_counts()

99.0    1158
2.0      377
1.0      346
Name: labels, dtype: int64

In [22]:
X

,max freq ch 0,max freq ch 1,max freq ch 2,max freq ch 3,max_freq_mean,max_freq_mode,max_freq_median,max_freq_quantile,max_freq_std,max_freq_var,...,freq_label_2_std,freq_label_1_var,freq_label_2_var,freq_label_1_entropy,freq_label_2_entropy,freq_label_1_skew,freq_label_2_skew,freq_label_1_kurtosis,freq_label_2_kurtosis,purness
0,13.066667,14.933333,14.666667,13.066667,13.933333,13.066667,13.500000,14.773333,7.630563e-01,17.499774,...,-0.705486,-1.263224,-1.410973,0.881121,0.893487,1.886106,1.470116,2.717115,1.250422,1
1,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,14.400000,1.776357e-15,20.480000,...,-0.287339,-0.518265,-0.574678,1.040429,1.024529,0.692378,0.425566,-0.305491,-1.011907,1
2,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,15.733333,0.000000e+00,24.448176,...,0.317052,-0.603647,0.634104,1.056579,1.018070,0.164519,0.177694,-1.179616,-1.399866,1
3,15.466667,15.466667,16.533333,16.266667,15.933333,15.466667,15.700000,16.373333,4.129223e-01,23.842956,...,0.055607,0.043921,0.111214,1.036543,1.064096,0.558361,0.479318,-0.479701,-0.789892,1
4,10.133333,18.933333,10.133333,10.133333,12.333333,10.133333,10.133333,14.973333,3.048409e+00,16.376103,...,-0.882854,-1.699296,-1.765708,0.954192,0.991516,1.504164,1.003931,1.444561,-0.160364,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,13.333333,11.200000,11.200000,13.600000,12.333333,11.200000,11.766667,13.440000,9.974715e-01,13.410402,...,-0.139515,-0.114171,-0.279030,1.048016,1.024135,0.702752,1.812317,-0.150278,3.428484,1
2610,13.600000,19.733333,12.533333,13.866667,14.933333,12.533333,13.733333,16.853333,2.298074e+00,19.942456,...,0.206982,0.263408,0.413963,1.000522,0.966982,1.245700,1.492514,1.644790,1.751985,1
2611,13.866667,16.800000,13.866667,13.866667,14.600000,13.866667,13.866667,15.480000,1.016136e+00,18.945899,...,0.090320,0.850828,0.180639,0.862530,0.908462,2.201012,0.675147,4.127730,-1.328274,1
2612,12.533333,15.733333,12.533333,18.666667,14.866667,12.533333,13.700000,16.906667,2.144199e+00,19.617688,...,-0.394556,0.805450,-0.789111,0.836923,0.995172,2.003319,1.447157,3.372156,2.193822,1


In [75]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
target_names = ['1', '2','99']


dc = DummyClassifier(strategy='uniform')
dc.fit(X_train, y_train)
y_train_pred=dc.predict(X_train)
print(classification_report(y_train, y_train_pred ))

              precision    recall  f1-score   support

         1.0       0.18      0.33      0.24       346
         2.0       0.21      0.34      0.26       377
        99.0       0.62      0.36      0.45      1158

    accuracy                           0.35      1881
   macro avg       0.34      0.34      0.32      1881
weighted avg       0.46      0.35      0.37      1881



In [76]:
y_val_pred = dc.predict(X_val)
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.21      0.33      0.25        48
         2.0       0.23      0.43      0.30        53
        99.0       0.55      0.28      0.37       161

    accuracy                           0.32       262
   macro avg       0.33      0.35      0.31       262
weighted avg       0.42      0.32      0.33       262



In [77]:
y_test_pred = dc.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.21      0.39      0.28        87
         2.0       0.19      0.33      0.24        94
        99.0       0.58      0.29      0.39       290

    accuracy                           0.32       471
   macro avg       0.33      0.34      0.30       471
weighted avg       0.43      0.32      0.34       471



In [26]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

OVR=OneVsRestClassifier(LinearSVC(random_state=0))
OVR.fit(X_train, y_train)
y_train_pred = OVR.predict(X_train)
y_val_pred = OVR.predict(X_val)

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [27]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_train_pred))


              precision    recall  f1-score   support

         1.0       0.93      0.08      0.14       346
         2.0       0.48      0.40      0.44       377
        99.0       0.67      0.89      0.77      1158

    accuracy                           0.64      1881
   macro avg       0.70      0.46      0.45      1881
weighted avg       0.68      0.64      0.59      1881



In [28]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       1.00      0.04      0.08        48
         2.0       0.45      0.38      0.41        53
        99.0       0.66      0.88      0.75       161

    accuracy                           0.63       262
   macro avg       0.70      0.43      0.42       262
weighted avg       0.68      0.63      0.56       262



In [29]:
y_test_pred = OVR.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       1.00      0.07      0.13        87
         2.0       0.56      0.46      0.50        94
        99.0       0.68      0.91      0.78       290

    accuracy                           0.66       471
   macro avg       0.75      0.48      0.47       471
weighted avg       0.71      0.66      0.60       471



from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
cm=confusion_matrix(y_test,y_test_pred)
print('matriz de confusion test')
print(cm)
#tn, fp, fn, tp = confusion_matrix(y_test,y_test_pred).ravel()
print('matriz de confusion test')
plot_confusion_matrix(OVR.fit(X_train, y_train), X_test, y_test,
                                 display_labels=target_names,
                                 cmap=plt.cm.Blues,
                                 normalize='true')

In [30]:
from sklearn.multiclass import OneVsOneClassifier
OVO=OneVsOneClassifier(LinearSVC(random_state=0))
OVO.fit(X_train, y_train)
y_train_pred = OVO.predict(X_train)
y_val_pred = OVO.predict(X_val)

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [31]:
print(classification_report(y_val, y_val_pred))

              precision    recall  f1-score   support

         1.0       0.39      0.71      0.50        48
         2.0       0.34      0.75      0.47        53
        99.0       0.84      0.30      0.44       161

    accuracy                           0.47       262
   macro avg       0.52      0.59      0.47       262
weighted avg       0.66      0.47      0.46       262



In [32]:
y_test_pred = OVO.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         1.0       0.41      0.80      0.55        87
         2.0       0.35      0.78      0.49        94
        99.0       0.81      0.27      0.40       290

    accuracy                           0.47       471
   macro avg       0.53      0.62      0.48       471
weighted avg       0.65      0.47      0.44       471



### Random forest

In [78]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
param = {
    'n_estimators':[100,150,200],
    'criterion':['gini','entropy'],
    #'max_depth':[25,20,None,10],
    'min_samples_split':[2,5,None]
}
param_grid = {
  'n_estimators': [150],
  'max_features': [5, 7, 9],
  'max_depth'   : [None, 3, 10, 20],
  'criterion'   : ['gini', 'entropy']
}
#cv = GridSearchCV(estimator=rfc,param_grid=param, scoring='f1_micro',n_jobs=-1,cv=5, verbose=1)
rfc.fit(X_train, y_train)
y_train_pred = rfc.predict(X_train)
y_val_pred = rfc.predict(X_val)
y_test_pred = rfc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       346
         2.0       1.00      1.00      1.00       377
        99.0       1.00      1.00      1.00      1158

    accuracy                           1.00      1881
   macro avg       1.00      1.00      1.00      1881
weighted avg       1.00      1.00      1.00      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.73      0.33      0.46        48
         2.0       0.65      0.28      0.39        53
        99.0       0.68      0.92      0.78       161

    accuracy                           0.68       262
   macro avg       0.69      0.51      0.54       262
weighted avg       0.68      0.68      0.64       262

reporte test
              precision    recall  f1-score   support

         1.0       0.66      0.38      0.48        87
         2.0       0.69      0.31      0.43        94
        99.0       0.69      

In [81]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(rfc.feature_importances_,3)*100})
importances = importances.sort_values('importance',ascending=False)
importances

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,feature,importance
2,max freq ch 2,3.6
41,sum_weighted_label_2_log,3.0
40,sum_weighted_label_1_log,2.8
28,freq_label_1_skew,2.7
20,sum_weighted_label_1,2.7
21,sum_weighted_label_2,2.6
30,freq_label_1_kurtosis,2.6
11,max_freq_skew,2.5
26,freq_label_1_entropy,2.4
46,freq_label_1_entropy_log,2.3


In [34]:
best_tree_clf = cv.best_estimator_
best_tree_clf

NameError: name 'cv' is not defined

In [ ]:
cv.best_params_

### Support vector machine

In [35]:
from sklearn.svm import SVC

svc_model = SVC(gamma='auto',random_state=100)
svc_model.fit(X_train, y_train)

y_train_pred = svc_model.predict(X_train)
y_val_pred = svc_model.predict(X_val)
y_test_pred = svc_model.predict(X_test)

print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.83      0.40      0.54       346
         2.0       0.86      0.31      0.46       377
        99.0       0.71      0.96      0.82      1158

    accuracy                           0.73      1881
   macro avg       0.80      0.56      0.61      1881
weighted avg       0.76      0.73      0.69      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.71      0.31      0.43        48
         2.0       0.67      0.23      0.34        53
        99.0       0.68      0.94      0.79       161

    accuracy                           0.68       262
   macro avg       0.69      0.49      0.52       262
weighted avg       0.68      0.68      0.63       262

reporte test
              precision    recall  f1-score   support

         1.0       0.72      0.32      0.44        87
         2.0       0.64      0.22      0.33        94
        99.0       0.67      

### gradient boosting

In [88]:

from sklearn.ensemble import GradientBoostingClassifier

# GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=10)
gbc.fit(X_train[features], y_train)
y_train_pred = gbc.predict(X_train[features])
y_val_pred = gbc.predict(X_val[features])
y_test_pred = gbc.predict(X_test[features])
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.84      0.58      0.68       346
         2.0       0.91      0.52      0.66       377
        99.0       0.78      0.96      0.86      1158

    accuracy                           0.80      1881
   macro avg       0.84      0.69      0.74      1881
weighted avg       0.82      0.80      0.79      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.67      0.42      0.51        48
         2.0       0.56      0.36      0.44        53
        99.0       0.70      0.86      0.77       161

    accuracy                           0.68       262
   macro avg       0.64      0.55      0.57       262
weighted avg       0.67      0.68      0.66       262

reporte test
              precision    recall  f1-score   support

         1.0       0.71      0.47      0.57        87
         2.0       0.72      0.44      0.54        94
        99.0       0.73      

In [84]:
importances = pd.DataFrame({'feature':X_train.columns,'importance':np.round(gbc.feature_importances_,3)*100})
importances = importances.sort_values('importance',ascending=False)
importances.head(15)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,feature,importance
2,max freq ch 2,19.9
1,max freq ch 1,4.9
40,sum_weighted_label_1_log,4.7
20,sum_weighted_label_1,3.9
28,freq_label_1_skew,3.7
11,max_freq_skew,3.5
0,max freq ch 0,2.9
41,sum_weighted_label_2_log,2.7
30,freq_label_1_kurtosis,2.7
48,purness,2.5


In [87]:
features=importances.head(15).feature.tolist()

In [53]:
from sklearn.pipeline import make_pipeline

gbc = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=100))
# GradientBoostingClassifier
#gbc = GradientBoostingClassifier(random_state=10)
gbc.fit(X_train, y_train)
y_train_pred = gbc.predict(X_train)
y_val_pred = gbc.predict(X_val)
y_test_pred = gbc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

<IPython.core.display.Javascript object>

reporte train
              precision    recall  f1-score   support

         1.0       0.87      0.66      0.75       346
         2.0       0.91      0.58      0.71       377
        99.0       0.81      0.97      0.88      1158

    accuracy                           0.83      1881
   macro avg       0.86      0.73      0.78      1881
weighted avg       0.84      0.83      0.82      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.71      0.42      0.53        48
         2.0       0.61      0.38      0.47        53
        99.0       0.70      0.88      0.78       161

    accuracy                           0.69       262
   macro avg       0.67      0.56      0.59       262
weighted avg       0.68      0.69      0.67       262

reporte test
              precision    recall  f1-score   support

         1.0       0.64      0.47      0.54        87
         2.0       0.64      0.39      0.49        94
        99.0       0.72      

### SGDC

In [37]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
sgdc = SGDClassifier(random_state=100)
sgdc.fit(X_train, y_train)
y_train_pred = sgdc.predict(X_train)
y_val_pred = sgdc.predict(X_val)
y_test_pred = sgdc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.76      0.31      0.44       346
         2.0       0.42      0.53      0.47       377
        99.0       0.72      0.79      0.75      1158

    accuracy                           0.65      1881
   macro avg       0.63      0.54      0.55      1881
weighted avg       0.67      0.65      0.64      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.74      0.29      0.42        48
         2.0       0.42      0.51      0.46        53
        99.0       0.70      0.78      0.74       161

    accuracy                           0.63       262
   macro avg       0.62      0.53      0.54       262
weighted avg       0.65      0.63      0.62       262

reporte test
              precision    recall  f1-score   support

         1.0       0.80      0.40      0.53        87
         2.0       0.43      0.52      0.47        94
        99.0       0.75      

### logistic regresion

In [38]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=100)
lrc.fit(X_train, y_train)
y_train_pred = lrc.predict(X_train)
y_val_pred = lrc.predict(X_val)
y_test_pred = lrc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.66      0.38      0.48       346
         2.0       0.61      0.28      0.38       377
        99.0       0.69      0.90      0.78      1158

    accuracy                           0.68      1881
   macro avg       0.65      0.52      0.55      1881
weighted avg       0.67      0.68      0.64      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.77      0.42      0.54        48
         2.0       0.60      0.34      0.43        53
        99.0       0.69      0.89      0.78       161

    accuracy                           0.69       262
   macro avg       0.69      0.55      0.58       262
weighted avg       0.69      0.69      0.67       262

reporte test
              precision    recall  f1-score   support

         1.0       0.68      0.48      0.56        87
         2.0       0.67      0.37      0.48        94
        99.0       0.71      

/home/ariel/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### decision tree

In [39]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=100)
dtc.fit(X_train, y_train)
y_train_pred = dtc.predict(X_train)
y_val_pred = dtc.predict(X_val)
y_test_pred = dtc.predict(X_test)
print('reporte train')
print(classification_report(y_train, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       346
         2.0       1.00      1.00      1.00       377
        99.0       1.00      1.00      1.00      1158

    accuracy                           1.00      1881
   macro avg       1.00      1.00      1.00      1881
weighted avg       1.00      1.00      1.00      1881

reporte validacion
              precision    recall  f1-score   support

         1.0       0.33      0.35      0.34        48
         2.0       0.43      0.53      0.47        53
        99.0       0.69      0.62      0.65       161

    accuracy                           0.55       262
   macro avg       0.48      0.50      0.49       262
weighted avg       0.57      0.55      0.56       262

reporte test
              precision    recall  f1-score   support

         1.0       0.41      0.45      0.43        87
         2.0       0.41      0.49      0.45        94
        99.0       0.72      

### balanceo de datos


In [47]:
#pruebo balanceando el dataset sacando muestras 99
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
over_sampler = SMOTE(k_neighbors=2)
X_res, y_res = over_sampler.fit_resample(X_train, y_train)
#print(f"Training target statistics: {Counter(y_res)}")
#print(f"Testing target statistics: {Counter(y_test)}")

In [48]:
y_res.value_counts()

99.0    1158
2.0     1158
1.0     1158
Name: labels, dtype: int64

In [52]:
from sklearn.ensemble import GradientBoostingClassifier
# GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=10)
gbc.fit(X_res, y_res)
y_train_pred = gbc.predict(X_res)
y_val_pred = gbc.predict(X_val)
y_test_pred = gbc.predict(X_test)
print('reporte train')
print(classification_report(y_res, y_train_pred))
print('reporte validacion')
print(classification_report(y_val, y_val_pred))
print('reporte test')
print(classification_report(y_test, y_test_pred))

reporte train
              precision    recall  f1-score   support

         1.0       0.86      0.91      0.88      1158
         2.0       0.85      0.88      0.86      1158
        99.0       0.88      0.79      0.83      1158

    accuracy                           0.86      3474
   macro avg       0.86      0.86      0.86      3474
weighted avg       0.86      0.86      0.86      3474

reporte validacion
              precision    recall  f1-score   support

         1.0       0.45      0.60      0.52        48
         2.0       0.53      0.58      0.55        53
        99.0       0.74      0.64      0.69       161

    accuracy                           0.62       262
   macro avg       0.57      0.61      0.59       262
weighted avg       0.64      0.62      0.63       262

reporte test
              precision    recall  f1-score   support

         1.0       0.50      0.63      0.56        87
         2.0       0.54      0.57      0.56        94
        99.0       0.78      